In [1]:
from obspy import read, Stream
import os
import sys
from glob import glob
from obspy.signal.trigger import recursive_sta_lta
from obspy.signal.trigger import coincidence_trigger
import pandas as pd
import numpy as np

In [2]:
# Load ERT data
ert_surveys_file = "../survey_times_ERT.csv"
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
ert_surveys = pd.read_csv(ert_surveys_file, parse_dates=["time_local_start"], date_parser=dateparse)
ert_surveys["time_local_start"] = ert_surveys["time_local_start"].dt.tz_localize("America/Edmonton",
                                                                                 ambiguous="infer")
ert_surveys["time_utc_start"] = ert_surveys["time_local_start"].dt.tz_convert(None)
ert_surveys["time_utc_end"] = ert_surveys["time_utc_start"] + pd.Timedelta(20, unit="m")
ert_surveys["time_utc_end"] = pd.to_datetime(ert_surveys["time_utc_end"])
ert_surveys["time_utc_start"] = pd.to_datetime(ert_surveys["time_utc_start"])


/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
import os

n = 0

DOM_PERIOD = 1 / 30.
for sta in [2*DOM_PERIOD, 3*DOM_PERIOD]:
    for lta in [5*DOM_PERIOD, 6*DOM_PERIOD, 7*DOM_PERIOD, 8*DOM_PERIOD, 9*DOM_PERIOD, 10*DOM_PERIOD]:
        for on_thresh in [3, 4, 6, 8, 10]:
            
            job_file = os.path.join("inslurm","job%d.slurm" % n)
            
            with open(job_file, "w") as fh:
                fh.writelines("#!/bin/bash\n")
                fh.writelines("#SBATCH --time=1:00:00\n")
                fh.writelines("#SBATCH --mem=25G\n")
                fh.writelines("#SBATCH --partition=geo,glados12\n")
                fh.writelines("#SBATCH --cpus-per-task=1\n")
                fh.writelines("#SBATCH --nodes=1\n")
                fh.writelines("#SBATCH --output=""outslurm/slurm-%A_%a.out""\n")
                fh.writelines("python detect.py %f %f %f\n" % (sta, lta, on_thresh))

            os.system("sbatch %s" % job_file)
            
            n += 1
            
# Marie's values
for sta in [1, 5, 10]:
    for lta in [100, 500, 1000, 3500]:
        for on_thresh in [3, 4, 6, 8, 10]:
            
            job_file = os.path.join("inslurm","job%d.slurm" % n)
            
            with open(job_file, "w") as fh:
                fh.writelines("#!/bin/bash\n")
                fh.writelines("#SBATCH --time=1:00:00\n")
                fh.writelines("#SBATCH --mem=25G\n")
                fh.writelines("#SBATCH --partition=geo,glados12\n")
                fh.writelines("#SBATCH --cpus-per-task=1\n")
                fh.writelines("#SBATCH --nodes=1\n")
                fh.writelines("#SBATCH --output=""outslurm/slurm-%A_%a.out""\n")
                fh.writelines("python detect.py %f %f %f\n" % (sta, lta, on_thresh))

            os.system("sbatch %s" % job_file)
            
            n += 1

In [42]:
from glob import glob
import numpy as np
import pandas as pd
from obspy import UTCDateTime

detections = np.array([])

flist = glob("*.list")
for file in flist:
        
    M = np.loadtxt(file, delimiter=",", dtype=UTCDateTime, skiprows=1, usecols=0)
    if M.shape[0] > 0:
        detections = np.concatenate((detections, M))

detections = np.unique(detections)

convert2utcdatetime = lambda t: UTCDateTime(t)
vfunc = np.vectorize(convert2utcdatetime)
detections = vfunc(detections)


/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.20_on8.0_off4.0.list"
  # This is added back by InteractiveShellApp.init_path()
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.23_on8.0_off4.0.list"
  # This is added back by InteractiveShellApp.init_path()
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.27_on4.0_off2.0.list"
  # This is added back by InteractiveShellApp.init_path()
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.23_on10.0_off5.0.list"
  # This is added back by InteractiveShellApp.init_path()
/home/g

/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.23_on6.0_off3.0.list"
  # This is added back by InteractiveShellApp.init_path()
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.33_on8.0_off4.0.list"
  # This is added back by InteractiveShellApp.init_path()


In [27]:
from obspy import Stream, read

DATA_DIR_ROOT = "/home/gilbert_lab/cami_frs/borehole_data/sac_daily_nez_500Hz/"
day_folder = "20200310"
stream = Stream()
for sta in ['G2', 'G4', 'G14', 'G6', 'G8', 'G11', 'G16', 'G18']:
    stream += read(os.path.join(DATA_DIR_ROOT, day_folder, "*%s.*.sac" % sta))
print(stream)#.__str__(extended=True))

24 Trace(s) in Stream:

BH.G2..DPE | 2020-03-10T00:00:00.000000Z - 2020-03-11T00:00:00.005273Z | 500.0 Hz, 43200001 samples
...
(22 other traces)
...
BH.G18..DPZ | 2020-03-10T00:00:00.000000Z - 2020-03-11T00:00:00.005273Z | 500.0 Hz, 43200001 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]


In [46]:
detections = np.unique(detections)
print(detections.shape)

for det in detections:
    print(det)
    d = stream.slice(starttime=det-0.5, endtime=det+0.5)
    d.write(os.path.join("waveforms", det.strftime("%Y%m%d%H%M%S.mseed")), format="MSEED")

(943,)
2020-03-10T00:08:19.000000Z
2020-03-10T00:08:20.000000Z
2020-03-10T00:08:49.000000Z
2020-03-10T01:01:49.000000Z
2020-03-10T01:01:57.000000Z
2020-03-10T01:01:58.000000Z
2020-03-10T01:02:02.000000Z
2020-03-10T01:02:03.000000Z
2020-03-10T01:02:18.000000Z
2020-03-10T01:07:18.000000Z
2020-03-10T01:07:19.000000Z
2020-03-10T01:10:32.000000Z
2020-03-10T01:10:33.000000Z
2020-03-10T01:11:22.000000Z
2020-03-10T01:13:56.000000Z
2020-03-10T01:15:01.000000Z
2020-03-10T01:16:08.000000Z
2020-03-10T01:16:09.000000Z
2020-03-10T01:19:16.000000Z
2020-03-10T01:25:22.000000Z
2020-03-10T01:28:56.000000Z
2020-03-10T01:39:05.000000Z
2020-03-10T01:40:32.000000Z
2020-03-10T01:41:44.000000Z
2020-03-10T01:42:20.000000Z
2020-03-10T01:43:51.000000Z
2020-03-10T01:47:22.000000Z
2020-03-10T01:48:34.000000Z
2020-03-10T01:48:35.000000Z
2020-03-10T01:48:36.000000Z
2020-03-10T01:48:38.000000Z
2020-03-10T01:48:39.000000Z
2020-03-10T01:48:40.000000Z
2020-03-10T01:48:42.000000Z
2020-03-10T01:48:43.000000Z
2020-03-10T01

2020-03-10T04:37:35.000000Z
2020-03-10T04:37:36.000000Z
2020-03-10T04:38:31.000000Z
2020-03-10T04:38:48.000000Z
2020-03-10T04:39:17.000000Z
2020-03-10T04:41:50.000000Z
2020-03-10T04:42:26.000000Z
2020-03-10T04:42:42.000000Z
2020-03-10T04:43:03.000000Z
2020-03-10T04:43:47.000000Z
2020-03-10T04:44:00.000000Z
2020-03-10T04:44:42.000000Z
2020-03-10T04:45:04.000000Z
2020-03-10T04:45:07.000000Z
2020-03-10T04:45:27.000000Z
2020-03-10T04:46:18.000000Z
2020-03-10T04:46:21.000000Z
2020-03-10T04:46:37.000000Z
2020-03-10T04:46:49.000000Z
2020-03-10T04:46:52.000000Z
2020-03-10T04:47:32.000000Z
2020-03-10T04:47:49.000000Z
2020-03-10T04:48:15.000000Z
2020-03-10T04:50:29.000000Z
2020-03-10T04:50:51.000000Z
2020-03-10T04:51:07.000000Z
2020-03-10T04:51:49.000000Z
2020-03-10T04:52:21.000000Z
2020-03-10T04:53:16.000000Z
2020-03-10T04:53:42.000000Z
2020-03-10T04:54:17.000000Z
2020-03-10T04:55:22.000000Z
2020-03-10T04:55:32.000000Z
2020-03-10T04:56:02.000000Z
2020-03-10T04:56:16.000000Z
2020-03-10T04:56:57.

2020-03-10T08:08:42.000000Z
2020-03-10T08:09:50.000000Z
2020-03-10T08:09:51.000000Z
2020-03-10T08:11:09.000000Z
2020-03-10T08:11:22.000000Z
2020-03-10T08:11:58.000000Z
2020-03-10T08:11:59.000000Z
2020-03-10T08:12:30.000000Z
2020-03-10T08:13:16.000000Z
2020-03-10T08:13:38.000000Z
2020-03-10T08:14:12.000000Z
2020-03-10T08:14:24.000000Z
2020-03-10T08:16:54.000000Z
2020-03-10T08:18:17.000000Z
2020-03-10T08:20:06.000000Z
2020-03-10T08:20:49.000000Z
2020-03-10T08:20:51.000000Z
2020-03-10T08:22:26.000000Z
2020-03-10T08:22:27.000000Z
2020-03-10T08:23:57.000000Z
2020-03-10T08:25:22.000000Z
2020-03-10T08:26:43.000000Z
2020-03-10T08:27:21.000000Z
2020-03-10T08:32:03.000000Z
2020-03-10T08:35:08.000000Z
2020-03-10T08:37:10.000000Z
2020-03-10T08:38:10.000000Z
2020-03-10T08:39:36.000000Z
2020-03-10T08:40:46.000000Z
2020-03-10T08:40:52.000000Z
2020-03-10T08:41:40.000000Z
2020-03-10T08:44:56.000000Z
2020-03-10T08:45:06.000000Z
2020-03-10T08:48:21.000000Z
2020-03-10T08:49:07.000000Z
2020-03-10T08:51:35.

2020-03-10T15:32:54.000000Z
2020-03-10T15:32:55.000000Z
2020-03-10T15:32:56.000000Z
2020-03-10T15:32:57.000000Z
2020-03-10T15:32:58.000000Z
2020-03-10T15:32:59.000000Z
2020-03-10T15:33:00.000000Z
2020-03-10T15:34:28.000000Z
2020-03-10T15:36:13.000000Z
2020-03-10T18:35:11.000000Z
2020-03-10T18:35:14.000000Z
2020-03-10T18:38:31.000000Z
2020-03-10T20:18:45.000000Z
2020-03-10T20:18:47.000000Z
2020-03-10T20:18:48.000000Z
2020-03-10T20:19:16.000000Z
2020-03-10T21:06:32.000000Z
2020-03-10T21:26:55.000000Z
2020-03-10T21:30:30.000000Z
2020-03-10T21:38:49.000000Z
2020-03-10T21:38:50.000000Z
2020-03-10T22:08:01.000000Z
2020-03-10T22:08:06.000000Z
2020-03-10T22:08:10.000000Z
2020-03-10T22:08:11.000000Z
2020-03-10T22:08:18.000000Z
2020-03-10T22:08:21.000000Z
2020-03-10T22:11:32.000000Z
2020-03-10T22:11:33.000000Z
2020-03-10T22:11:34.000000Z
2020-03-10T22:11:35.000000Z
2020-03-10T22:11:36.000000Z
2020-03-10T22:11:37.000000Z
2020-03-10T22:11:38.000000Z
2020-03-10T22:11:39.000000Z
2020-03-10T22:11:40.

In [51]:
convert2utcdatetime = lambda t: UTCDateTime(t)
vfunc = np.vectorize(convert2utcdatetime)

flist = glob("*.list")
for file in flist:
    
    detections = np.loadtxt(file, delimiter=",", dtype=UTCDateTime, skiprows=1, usecols=0)
    if detections.shape[0] > 0:
        print(file)        
        detections = vfunc(detections)

        ngood = 0
        nbad = 0
        for det in detections:
            
            fname = det.strftime("%Y%m%d%H%M%S.mseed")
            if os.path.exists(os.path.join("bad", fname)):
                nbad += 1
            elif os.path.exists(os.path.join("good", fname)):
                ngood += 1
        print("\tntotal = %d\n\tngood = %d\n\tnbad = %d\n\tRate of false detections: %d" % (len(detections), ngood, nbad, 100*(nbad/(nbad+ngood))))

/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.20_on8.0_off4.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.23_on8.0_off4.0.list"
  import sys


detections_sta1.00_lta3500.00_on6.0_off3.0.list
	ntotal = 224
	ngood = 124
	nbad = 100
	Rate of false detections: 44
detections_sta5.00_lta1000.00_on8.0_off4.0.list
	ntotal = 67
	ngood = 49
	nbad = 18
	Rate of false detections: 26
detections_sta5.00_lta100.00_on10.0_off5.0.list
	ntotal = 38
	ngood = 27
	nbad = 11
	Rate of false detections: 28
detections_sta0.07_lta0.30_on3.0_off1.5.list
	ntotal = 605
	ngood = 316
	nbad = 289
	Rate of false detections: 47
detections_sta5.00_lta1000.00_on3.0_off1.5.list
	ntotal = 135
	ngood = 78
	nbad = 57
	Rate of false detections: 42
detections_sta10.00_lta1000.00_on4.0_off2.0.list
	ntotal = 69
	ngood = 49
	nbad = 20
	Rate of false detections: 28
detections_sta1.00_lta3500.00_on10.0_off5.0.list
	ntotal = 138
	ngood = 88
	nbad = 50
	Rate of false detections: 36
detections_sta10.00_lta500.00_on6.0_off3.0.list
	ntotal = 51
	ngood = 37
	nbad = 14
	Rate of false detections: 27
detections_sta5.00_lta1000.00_on4.0_off2.0.list
	ntotal = 106
	ngood = 63
	nbad =

/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.27_on4.0_off2.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.23_on10.0_off5.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.20_on3.0_off1.5.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.23_on3.0_off1.5.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.17_on8.0_off4.0.list"
  

	ntotal = 143
	ngood = 81
	nbad = 62
	Rate of false detections: 43
detections_sta0.07_lta0.33_on3.0_off1.5.list
	ntotal = 757
	ngood = 379
	nbad = 378
	Rate of false detections: 49
detections_sta1.00_lta500.00_on4.0_off2.0.list
	ntotal = 325
	ngood = 167
	nbad = 158
	Rate of false detections: 48
detections_sta1.00_lta1000.00_on6.0_off3.0.list
	ntotal = 206
	ngood = 124
	nbad = 82
	Rate of false detections: 39
detections_sta1.00_lta3500.00_on8.0_off4.0.list
	ntotal = 164
	ngood = 103
	nbad = 61
	Rate of false detections: 37
detections_sta1.00_lta1000.00_on10.0_off5.0.list


/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.20_on3.0_off1.5.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.20_on6.0_off3.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.27_on10.0_off5.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.30_on10.0_off5.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.27_on6.0_off3.0.list"
 

...

/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.27_on3.0_off1.5.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.23_on4.0_off2.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.17_on4.0_off2.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.20_on8.0_off4.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.23_on4.0_off2.0.list"
  i

	ntotal = 424
	ngood = 199
	nbad = 225
	Rate of false detections: 53
detections_sta5.00_lta500.00_on4.0_off2.0.list
	ntotal = 105
	ngood = 63
	nbad = 42
	Rate of false detections: 40
detections_sta0.07_lta0.23_on3.0_off1.5.list
	ntotal = 189
	ngood = 114
	nbad = 75
	Rate of false detections: 39
detections_sta1.00_lta1000.00_on3.0_off1.5.list
	ntotal = 420
	ngood = 199
	nbad = 221
	Rate of false detections: 52
detections_sta1.00_lta500.00_on8.0_off4.0.list
	ntotal = 167
	ngood = 103
	nbad = 64
	Rate of false detections: 38
detections_sta1.00_lta100.00_on10.0_off5.0.list
	ntotal = 139
	ngood = 86
	nbad = 53
	Rate of false detections: 38
detections_sta5.00_lta1000.00_on6.0_off3.0.list
	ntotal = 80
	ngood = 53
	nbad = 27
	Rate of false detections: 33
detections_sta5.00_lta3500.00_on6.0_off3.0.list
	ntotal = 81
	ngood = 56
	nbad = 25
	Rate of false detections: 30
detections_sta1.00_lta100.00_on4.0_off2.0.list
	ntotal = 330
	ngood = 174
	nbad = 156
	Rate of false detections: 47
detections_st

/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.30_on6.0_off3.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.27_on6.0_off3.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.17_on6.0_off3.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.17_on4.0_off2.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.30_on8.0_off4.0.list"
  i

	ntotal = 172
	ngood = 107
	nbad = 65
	Rate of false detections: 37
detections_sta5.00_lta3500.00_on10.0_off5.0.list
	ntotal = 59
	ngood = 44
	nbad = 15
	Rate of false detections: 25
detections_sta0.07_lta0.27_on3.0_off1.5.list
	ntotal = 433
	ngood = 236
	nbad = 197
	Rate of false detections: 45
detections_sta10.00_lta500.00_on8.0_off4.0.list
	ntotal = 37
	ngood = 25
	nbad = 12
	Rate of false detections: 32
detections_sta5.00_lta500.00_on8.0_off4.0.list
	ntotal = 66
	ngood = 47
	nbad = 19
	Rate of false detections: 28
detections_sta10.00_lta1000.00_on6.0_off3.0.list
	ntotal = 52
	ngood = 38
	nbad = 14
	Rate of false detections: 26
detections_sta5.00_lta3500.00_on3.0_off1.5.list
	ntotal = 137
	ngood = 80
	nbad = 57
	Rate of false detections: 41
detections_sta1.00_lta100.00_on6.0_off3.0.list
	ntotal = 222
	ngood = 135
	nbad = 87
	Rate of false detections: 39
detections_sta0.07_lta0.27_on4.0_off2.0.list
	ntotal = 2
	ngood = 0
	nbad = 2
	Rate of false detections: 100
detections_sta1.00_lta

/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.33_on6.0_off3.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.10_lta0.33_on8.0_off4.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.23_on6.0_off3.0.list"
  import sys
/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: loadtxt: Empty input file: "detections_sta0.07_lta0.33_on8.0_off4.0.list"
  import sys


str